In [ ]:
#from __future__ import print_function
#import os.path
from googleapiclient.discovery import build
#from google_auth_oauthlib.flow import InstalledAppFlow
#from google.auth.transport.requests import Request
#from google.oauth2.credentials import Credentials
from google.oauth2 import service_account


SCOPES = ['https://www.googleapis.com/auth/spreadsheets']


#this scopes are are for SQL service admin
#SCOPES = ['https://www.googleapis.com/auth/sqlservice.admin']


SERVICE_ACCOUNT_FILE = './keys.json'
creds = None
creds = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)



# The ID of spreadsheet.
SAMPLE_SPREADSHEET_ID = '1yxdrbPX5fE3sgDqmzhAv0Bcaos03wb_yuWGCpMKtYC4'



    
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    #if os.path.exists('token.json'):
        #creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    #if not creds or not creds.valid:
        #if creds and creds.expired and creds.refresh_token:
            #creds.refresh(Request())
        #else:
            #flow = InstalledAppFlow.from_client_secrets_file(
                #'credentials.json', SCOPES)
           # creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        #with open('token.json', 'w') as token:
            #token.write(creds.to_json())

service = build('sheets', 'v4', credentials=creds)

# Call the Sheets API
sheet = service.spreadsheets()

In [ ]:
#read = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,range="Sheet1!A1:G16").execute()
#values = read.get('values', [])
#print(read)


In [ ]:
import pandas as pd
df=pd.read_csv('.\query_result_2021-04-13T18_01_02.361409Z.csv')

df

In [ ]:
import pandas as pd
df=pd.read_json('.\query_result_2021-04-19T11_25_04.285261Z.json')
df=df.values.tolist()
df

In [ ]:
data=[["AAAA",3049709],["BBB",54549709],["CAAA",9],["DFFF",709]]
write = sheet.values().update(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                              range="Sheet1", valueInputOption="RAW", body={"values":df}).execute()

In [ ]:
import datetime
import pandas as pd
date = datetime.date.today()
def create_sheet(title, data):
    sheets_service = build("sheets", "v4", credentials=creds)
    sheets = sheets_service.spreadsheets()

    # Body of create method with a Spreadsheet(https://developers.google.com/sheets/api/reference/rest/v4/spreadsheets#Spreadsheet) instance
    create_body = {"properties": {"title": f"{title} {date.today()}"},
                   "sheets": list(map(lambda d: {"properties": {"title": d.get("title")}}, data))}
    res = sheets.create(body=create_body).execute()
    spreadsheet_id = res.get("SAMPLE_SPREADSHEET_ID")

    # Transform the DataFrame into a matrix of the columns and values
    def df_to_sheet(df):
        df_columns = [np.array(df.columns)]
        df_values = df.values.tolist()
        df_to_sheet = np.concatenate((df_columns, df_values)).tolist()
        return df_to_sheet

    update_body = {
        "valueInputOption": "RAW",
        "data": list(map(lambda d: {"range": d.get("title"), "values": df_to_sheet(d.get("df"))}, data))
    }

    sheets.values().batchUpdate(spreadsheetId=spreadsheet_id, body=update_body).execute()

    return res
data=df=pd.read_csv('.\query_result_2021-04-13T18_01_02.361409Z.csv')
res = create_sheet("Title", data=data)


def share_spreadsheet(spreadsheet_id, options, notify=False):
    drive_service = build("drive", "v3", credentials=credentials)

    res = (
        drive_service.permissions()
        .create(
            fileId=spreadsheet_id,
            body=options,
            sendNotificationEmail=notify,
        )
        .execute()
    )

    return res




def create_and_share_spreadsheet(title, data, permissions):
    sheet = create_sheet(title, data)
    share_spreadsheet(sheet.get("spreadsheetId"), options=permissions.get(
        "info"), notify=permissions.get("notify"))

    return sheet.get("spreadsheetUrl")

In [ ]:
import gspread 

#create service account
gc = gspread.service_account(filename='./keys.json')

#open spreadsheet
sh = gc.open_by_key('1yxdrbPX5fE3sgDqmzhAv0Bcaos03wb_yuWGCpMKtYC4')
worksheet = sh.sheet1

#access the data from sheet
#res = worksheet.get_all_records()
#res = worksheet.get_all_values()
#res = worksheet.row_values(2)
#res = worksheet.col_values(2)
#res = worksheet.get('A2')
#res = worksheet.get('A2:C2')
print(res)

#insert data from sheet
user = ["Ponkoj",25,'SDFSD']
#worksheet.insert_row(user,6)
#worksheet.append_row(user)

#update specific cell values
#worksheet.update_cell(6,2,'Chandra') #update_cell(row,column,value)

#delete cells
#worksheet.delete_rows(3)
